# Training a new model with 1.2 million tweets


## Imports


In [0]:
!pip install ktrain
import pandas as pd
import ktrain 
from ktrain import text
from sklearn.model_selection import train_test_split
from os import path



     |████████████████████████████████| 174kB 5.1MB/s 
     |████████████████████████████████| 317kB 57.2MB/s 
     |████████████████████████████████| 112kB 64.1MB/s 
     |████████████████████████████████| 1.0MB 60.4MB/s 
     |████████████████████████████████| 245kB 68.8MB/s 
     |████████████████████████████████| 1.8MB 47.3MB/s 
  Created wheel for ktrain: filename=ktrain-0.6.2-cp36-none-any.whl size=223849 sha256=1a52f3eac276b9bf39b27ca7f5a56c97b2037dceed025dd4ad5130da01f13519
  Stored in directory: /root/.cache/pip/wheels/fa/85/7b/0ccbcd5f821b73d1436f90895cb46064c10aca1ca3d083d384
  Created wheel for keras-bert: filename=keras_bert-0.80.0-cp36-none-any.whl size=37923 sha256=14e86f19397661b634915c0ab16a78cdc3c372ce61d5a7833446d5b5a3e8b6d9
  Stored in directory: /root/.cache/pip/wheels/63/dc/87/3260cb91f3aa32c0f85c5375429a30c8fd988bbb48f5ee21b0
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=db8b326f7f9c11fa6a87cb2b329fae7796d273bcc1f9b8870e6

Using TensorFlow backend.


using Keras version: 2.2.4


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
df = pd.read_csv('/content/drive/My Drive/senti_dataset/tweet_shuffled.csv', index_col=0)
df

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,polarity,sentence
0,0,my god this book is like listening to a record...
1,0,need to go to bed now im wide awake too
2,0,seems like a lacking concentration today even ...
3,1,crochet liberation front featured in inside cr...
4,0,scoop: HASHTAG hasselhoff rushed to the hospit...
...,...,...
1599995,1,USER_MENTION USER_MENTION and USER_MENTION ar...
1599996,1,USER_MENTION hi newbies. come and tweet with ...
1599997,1,USER_MENTION i thought i was being slick
1599998,0,nooooooooooooo i cant hear max on the radio!!!


## Sampling and Preprocessing

In [0]:
BATCH_SIZE = 200000

def training_generator(df, BATCH_SIZE):
  x = []
  y = []  
  for row in df.itertuples():
    if int(row.Index) % BATCH_SIZE == 0:
      x = []
      y = []
    x.append(row.sentence)
    y.append(row.polarity)
    if int(row.Index + 1) % BATCH_SIZE == 0:
      yield x,y


In [0]:
predictor_path = '/content/drive/My Drive/senti_model/' + 'tweetPredictor_'

In [0]:
def return_last_predictor(predictor_path):
  for j in range(10):
    pred_full_path = predictor_path + str(j + 1)
    if(not path.exists(pred_full_path)):
      return j
    else:
      continue
  return j

In [0]:

# return last trained predictor
predictor_path = '/content/drive/My Drive/senti_model/' + 'tweetPredictor_'
i = return_last_predictor(predictor_path)
i

1

In [0]:
for j in range(i,10):  
  gen = training_generator(df, BATCH_SIZE)

  # get to the i'th data batch
  for k in range(i+1):
    x, y = next(gen)
    print(x[:5])

  # i'th batch of data is processed further  
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
  (x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=x_train, y_train=y_train,
                                                                       x_test=x_test, y_test=y_test,
                                                                       class_names=label_list,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=35000,
                                                                       lang='en')
  x = None
  y = None
  if(i):
    predictor_file = predictor_path + str(i)
    pred = ktrain.load_predictor(predictor_file)
    model = pred.model
  learner = ktrain.get_learner(model, train_data=(x_train, y_train), batch_size=6)
  learner.fit_onecycle(2e-5, 1)
  predictor = ktrain.get_predictor(learner.model, preproc, )
  predictor_path = '/content/drive/My Drive/senti_model/' + 'tweetPredictor_' + str(i + 1)
  predictor.save(predictor_path)

['my god this book is like listening to a record on repeat 170,000 times. i get it! aid to africa hasnt &amp; wont help. shut the fuck up! ', 'need to go to bed now im wide awake too', 'seems like a lacking concentration today even though has to finish up her work asap ', 'crochet liberation front featured in inside crochet (uk) &quot;on the net&quot; section URL ', 'scoop: HASHTAG hasselhoff rushed to the hospital again 4 alcohol poisoning.daughter saves his life again. what happened 2 our knight rider ']
['sipping a delicious cup of tea ', ' USER_MENTION ooo sounds like an awesome prize what do you have to do? ', 'cant believe susan boyle lost! ', 'enjoying the weekend with my baby doll ', 'boo to no internet due to the storm ']
['my god this book is like listening to a record on repeat 170,000 times. i get it! aid to africa hasnt &amp; wont help. shut the fuck up! ', 'need to go to bed now im wide awake too', 'seems like a lacking concentration today even though has to finish up her

In [0]:
next_x, next_y = next(gen)

,polarity,sentence
0,0,my god this book is like listening to a record...
1,0,need to go to bed now im wide awake too
2,0,seems like a lacking concentration today even ...
3,1,crochet liberation front featured in inside cr...
4,0,scoop: HASHTAG hasselhoff rushed to the hospit...


In [0]:
# let's start from here

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=x_train, y_train=y_train,
                                                                       x_test=x_test, y_test=y_test,
                                                                       class_names=label_list,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=35000,
                                                                       lang='en')


preprocessing train...
language: en


preprocessing test...
language: en


In [0]:
print(x[:30])
print(y[:30])

['i cant believe the shaman drum bookshop is closing ', ' USER_MENTION i got one and like it', ' USER_MENTION thanks ', ' USER_MENTION thanks for the HASHTAG ! ', ' USER_MENTION nice. i miss the bay area hope you have a great monday and week!', 'i cant wait to see the new moon trailer tomorrow! omg sooo excited!', 'getting ready for the fiestaaa!! ', 'expects a boring day ', 'omg i just looked at my feet and they are a mess. dont wear shoes too small to go for a walk in the sun. ouch ', ' headed to the city for the last photoshoot in this weeks series. ill be under for the rest of the day but ill be back tomorrow!!', 'the only person you can trust is yourself.dont argue dont give me the usual bs.its my opinion &amp; it works for me. ', ' USER_MENTION its so nice. ', 'at gutted can see USER_MENTION tomorow ', 'my throat is killiinggg me not happy jan', ' USER_MENTION - thank you ', 'loneliness miss u so much t__t', ' USER_MENTION aww thanks baby! i love you ', 'bouncing on the mattress.

In [0]:
# unset the original data
x = None
y = None

In [0]:
# you can disregard the deprecation warnings arising from using Keras 2.2.4 with TensorFlow 1.14.
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model, train_data=(x_train, y_train), batch_size=6)

Is Multi-Label? False
maxlen is 350
done.


In [0]:
learner.fit_onecycle(2e-5, 1)
predictor = ktrain.get_predictor(learner.model, preproc, )
predictor.save('/content/drive/My Drive/senti_model/tweetPredictor_firstepoch')
learner.fit_onecycle(2e-5, 1)
predictor = ktrain.get_predictor(learner.model, preproc, )
predictor.save('/content/drive/My Drive/senti_model/tweetPredictor_secondepoch')




begin training using onecycle policy with max lr of 2e-05...
Epoch 1/1
 57522/160000 [=========>....................] - ETA: 5:09:32 - loss: 0.4213 - acc: 0.8077

In [0]:
learner.validate(val_data=(x_test, y_test), class_names=["0","1"])

              precision    recall  f1-score   support

           0       0.85      0.86      0.86     20000
           4       0.86      0.85      0.85     20000

    accuracy                           0.85     40000
   macro avg       0.85      0.85      0.85     40000
weighted avg       0.85      0.85      0.85     40000



array([[17256,  2744],
       [ 3098, 16902]])

In [0]:
predictor = ktrain.get_predictor(learner.model, preproc, )

In [0]:
re_predictor.get_classes()

[0, 4]

In [0]:
predictor.save('/content/drive/My Drive/senti_model/tweetPredictor_twoepoch')

In [0]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

Gen RAM Free: 11.7 GB  | Proc size: 2.0 GB
GPU RAM Free: 10283MB | Used: 1158MB | Util  10% | Total 11441MB


In [0]:
re_predictor = ktrain.load_predictor('/content/drive/My Drive/senti_model/tweetPredictor_oneepoch')

In [0]:
preds = re_predictor.predict("we should not kill immigrants", return_proba=True)
print("negative result:  %f" %(preds[0]))
print("positive result:  %f" %(preds[]))

negative result:  0.794513
positive result:  0.000000


In [0]:
reloaded_predictor.predict("", return_proba=True)

NameError: ignored

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# Merging of datasets


In [0]:
import os
import tensorflow as tf
import re
import pandas as pd

# Load all files from a directory in a DataFrame.
def load_directory_data(directory):
  data = {}
  data["sentence"] = []
  data["sentiment"] = []
  for file_path in os.listdir(directory):
    with tf.gfile.GFile(os.path.join(directory, file_path), "r") as f:
      data["sentence"].append(f.read())
      data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
  return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(directory):
  pos_df = load_directory_data(os.path.join(directory, "pos"))
  neg_df = load_directory_data(os.path.join(directory, "neg"))
  pos_df["polarity"] = 1
  neg_df["polarity"] = 0
  return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

# Download and process the dataset files.
def download_and_load_datasets(force_download=False):
  dataset = tf.keras.utils.get_file(
      fname="aclImdb.tar.gz", 
      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
      extract=True)
  
  train_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                       "aclImdb", "train"))
  test_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                      "aclImdb", "test"))
  
  return train_df, test_df

In [0]:
train, test = download_and_load_datasets()

84131840/84125825 [==============================] - 7s 0us/step


In [0]:
x_train = train.sentence
y_train = train.polarity
x_test = test.sentence
y_test = test.polarity



In [0]:
import numpy as np
x = pd.concat([x_train, x_test]).reset_index()
x = x.drop(['index'], axis=1)
x

y = pd.concat([y_train, y_test]).reset_index()
y = y.drop(['index'], axis=1)
y
z = x.join(y)
z
z.to_csv('/content/drive/My Drive/senti_dataset/amazon_reviews.csv')

In [0]:
df = pd.read_csv('/content/drive/My Drive/senti_dataset/train_preprocessed_0and1.csv', header=None)
small_df = df.sample(n=200000)
small_df

,0,1,2
941864,941865,1,HASHTAG psst you ya you ive got tweet. all yo...
690066,690067,0,USER_MENTION damn. my parents wont let me
1104253,1104254,1,USER_MENTION USER_MENTION hmmm mustve missed ...
408070,408071,0,is gettin so frustrated designing tattoos but ...
1062394,1062395,1,USER_MENTION yeah it may smell funky but its ...
...,...,...,...
394804,394805,0,getting my study on for my final at 3. i need ...
365926,365927,0,last night at the beach.
457262,457263,0,USER_MENTION goodnight.
1535553,1535554,1,USER_MENTION we asked her if she had a name a...


In [0]:
small_df = small_df.rename(columns={1: "polarity", 2: "sentence"})

In [0]:
small_df = small_df.reset_index()

In [0]:
small_df = small_df.drop(['index'], axis=1)
small_df

,polarity,sentence
0,1,HASHTAG psst you ya you ive got tweet. all yo...
1,0,USER_MENTION damn. my parents wont let me
2,1,USER_MENTION USER_MENTION hmmm mustve missed ...
3,0,is gettin so frustrated designing tattoos but ...
4,1,USER_MENTION yeah it may smell funky but its ...
...,...,...
199995,0,getting my study on for my final at 3. i need ...
199996,0,last night at the beach.
199997,0,USER_MENTION goodnight.
199998,1,USER_MENTION we asked her if she had a name a...


In [0]:
total = small_df.append(z)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [0]:
total = total.reset_index()
total = total.drop(['index'], axis=1)
total
total.to_csv('/content/drive/My Drive/senti_dataset/merged_dataset.csv')

# Shuffling of tweets dataset


In [0]:
df = pd.read_csv('/content/drive/My Drive/senti_dataset/train_preprocessed_0and1.csv', header=None)
df = df.drop([0], axis=1)
df = df.sample(frac=1).reset_index(drop=True)
df = df.rename(columns={1: "polarity", 2: "sentence"})
df


,polarity,sentence
0,0,my god this book is like listening to a record...
1,0,need to go to bed now im wide awake too
2,0,seems like a lacking concentration today even ...
3,1,crochet liberation front featured in inside cr...
4,0,scoop: HASHTAG hasselhoff rushed to the hospit...
...,...,...
1599995,1,USER_MENTION USER_MENTION and USER_MENTION ar...
1599996,1,USER_MENTION hi newbies. come and tweet with ...
1599997,1,USER_MENTION i thought i was being slick
1599998,0,nooooooooooooo i cant hear max on the radio!!!


In [0]:
df.to_csv('/content/drive/My Drive/senti_dataset/tweet_shuffled.csv')